
# <p style="text-align: center;">Large Weather and Decrees</p>

In [1]:
#!pip install cartopy
#!pip install imageio

## Import libraries

In [2]:
import pandas as pd
import numpy as np

import glob
import os

import requests

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import zipfile

import xarray as xr

import folium
from folium.plugins import MarkerCluster

import pandas as pd
from geopy.geocoders import Nominatim

import geopandas as gpd
import cartopy.crs as ccrs

import imageio

from sklearn.neighbors import KDTree
from sklearn.neighbors import BallTree

from sklearn.preprocessing import RobustScaler

## Set parameters

In [3]:
decree_filename_base = 'arrete_'
decrees_folder_name = './../../data/raw/decrees'
communes_folder_name = './../../data/raw/opendatasoft'
weather_folder_name = './../../data/raw/weather/era5'
weather_ncfiles_folder_name = './../../data/raw/weather/era5/ncfiles2'
processed_data_folder_name = './../../data/processed'
output_data_folder_name = './../../data/processed/output'
decrees_filename = 'decrees.parquet'
decrees_locations_filename = 'decrees_locations.parquet'
communes_csv_filename = 'correspondance-code-insee-code-postal-202410.csv'
weather_filename = 'weather2.parquet'
weather_shema = 'weather_shema2.csv'
weather_yearly_filename = 'weather2_yearly.parquet'
weather_yearly_shema = 'weather_shema2_yearly.csv'
drought_filename = 'drought.parquet'
drought_shema = 'drought_shema.csv'
drought_yearly_filename = 'drought_yearly.parquet'
drought_yearly_shema = 'drought_yearly_shema.csv'
#weather_zip_file = "193fcd51a8958175843ecbbcaba057c8.zip"

## Load Weather data

| Column    | Description                                      |
|-----------|--------------------------------------------------|
| date      | The date of the observation.                     |
| latitude  | The latitude coordinate of the observation point.|
| longitude | The longitude coordinate of the observation point.|
| number    | A unique identifier for the observation.         |
| expver    | Experiment version number.                       |
| u10       | 10-meter U-component of wind (eastward wind).    |
| v10       | 10-meter V-component of wind (northward wind).   |
| t2m       | 2-meter temperature (air temperature at 2 meters above the surface).|
| sp        | Surface pressure.                                |
| tp        | Total precipitation.                             |
| e         | Evaporation.                                     |
| sro       | Surface runoff.                                  |
| tcrw      | Total column water vapor.                        |
| stl1      | Soil temperature level 1.                        |
| stl2      | Soil temperature level 2.                        |
| slt       | Soil type.                                       |
| swvl1     | Volumetric soil water layer 1.                   |
| swvl2     | Volumetric soil water layer 2.                   |
| cvh       | High vegetation cover.                           |
| cvl       | Low vegetation cover.                            |
| tvh       | High vegetation type.                            |
| tvl       | Low vegetation type.                             |


In [4]:
df_yearly_weather = pd.read_parquet(os.path.join(processed_data_folder_name, weather_yearly_filename))

In [5]:
df_yearly_weather.head()

,latitude,longitude,year,t2m_z_sum,t2m_z_mean,tp_z_sum,tp_z_mean,e_z_sum,e_z_mean,pev_z_sum,...,stl1_sum,stl1_mean,stl2_sum,stl2_mean,slt_sum,slt_mean,swvl1_sum,swvl1_mean,swvl2_sum,swvl2_mean
0,42.5,-5.0,1940,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,...,131.224976,10.935414,131.221069,10.935089,36.0,3.0,3.491412,0.290951,3.582633,0.298553
1,42.5,-5.0,1941,0.000000,0.000000,-1.414214,-0.117851,1.414214,0.117851,-1.414214,...,130.369019,10.864085,129.770386,10.814198,36.0,3.0,3.291583,0.274299,3.486492,0.290541
2,42.5,-5.0,1942,2.590982,0.215915,0.948012,0.079001,-2.155737,-0.179645,-0.856798,...,132.005249,11.000438,131.522339,10.960195,36.0,3.0,3.473109,0.289426,3.584625,0.298719
3,42.5,-5.0,1943,3.948252,0.329021,-5.062950,-0.421913,-0.876471,-0.073039,-7.460276,...,143.968140,11.997345,143.213501,11.934459,36.0,3.0,3.147441,0.262287,3.267897,0.272325
4,42.5,-5.0,1944,-0.020748,-0.001729,-2.657244,-0.221437,5.106711,0.425559,-5.488278,...,133.577759,11.131480,133.382690,11.115224,36.0,3.0,2.850830,0.237569,2.922482,0.243540


In [6]:
df_yearly_weather.columns

Index(['latitude', 'longitude', 'year', 't2m_z_sum', 't2m_z_mean', 'tp_z_sum',
       'tp_z_mean', 'e_z_sum', 'e_z_mean', 'pev_z_sum', 'pev_z_mean',
       'stl1_z_sum', 'stl1_z_mean', 'stl2_z_sum', 'stl2_z_mean', 'slt_z_sum',
       'slt_z_mean', 'swvl1_z_sum', 'swvl1_z_mean', 'swvl2_z_sum',
       'swvl2_z_mean', 't2m_sum', 't2m_mean', 'tp_sum', 'tp_mean', 'e_sum',
       'e_mean', 'pev_sum', 'pev_mean', 'stl1_sum', 'stl1_mean', 'stl2_sum',
       'stl2_mean', 'slt_sum', 'slt_mean', 'swvl1_sum', 'swvl1_mean',
       'swvl2_sum', 'swvl2_mean'],
      dtype='object')

In [8]:
# Apply the schema to the loaded dataframe
df_yearly_weather = df_yearly_weather.astype(schema_yearly.to_dict())

NameError: name 'schema_yearly' is not defined

In [9]:
df_yearly_weather.dtypes

latitude        float64
longitude       float64
year              int32
t2m_z_sum       float64
t2m_z_mean      float64
tp_z_sum        float64
tp_z_mean       float64
e_z_sum         float64
e_z_mean        float64
pev_z_sum       float64
pev_z_mean      float64
stl1_z_sum      float64
stl1_z_mean     float64
stl2_z_sum      float64
stl2_z_mean     float64
slt_z_sum       float64
slt_z_mean      float64
swvl1_z_sum     float64
swvl1_z_mean    float64
swvl2_z_sum     float64
swvl2_z_mean    float64
t2m_sum         float32
t2m_mean        float32
tp_sum          float32
tp_mean         float32
e_sum           float32
e_mean          float32
pev_sum         float32
pev_mean        float32
stl1_sum        float32
stl1_mean       float32
stl2_sum        float32
stl2_mean       float32
slt_sum         float32
slt_mean        float32
swvl1_sum       float32
swvl1_mean      float32
swvl2_sum       float32
swvl2_mean      float32
dtype: object

In [10]:
print(df_yearly_weather.isna().sum())  # Count missing values

latitude           0
longitude          0
year               0
t2m_z_sum          0
t2m_z_mean      1925
tp_z_sum           0
tp_z_mean       1925
e_z_sum            0
e_z_mean        1925
pev_z_sum          0
pev_z_mean      1925
stl1_z_sum         0
stl1_z_mean     1925
stl2_z_sum         0
stl2_z_mean     1925
slt_z_sum          0
slt_z_mean      1925
swvl1_z_sum        0
swvl1_z_mean    1925
swvl2_z_sum        0
swvl2_z_mean    1925
t2m_sum            0
t2m_mean           0
tp_sum             0
tp_mean            0
e_sum              0
e_mean             0
pev_sum            0
pev_mean           0
stl1_sum           0
stl1_mean          0
stl2_sum           0
stl2_mean          0
slt_sum            0
slt_mean           0
swvl1_sum          0
swvl1_mean         0
swvl2_sum          0
swvl2_mean         0
dtype: int64


In [11]:
# Filter the DataFrame to exclude the year 1940
df_yearly_weather[df_yearly_weather['year'] != 1940].isna().sum().sum()




0

In [12]:
df_yearly_weather.drop(df_yearly_weather[df_yearly_weather['year'] == 1940].index, inplace=True)

In [13]:
df_yearly_weather.isna().sum().sum()

0

### Note

>There are no missing values in weather data because they come from monthly ERA5 simulations based on real data. only missing in 1940 du to rolling window for featured parameters.

## Import Decrees

In [14]:


df_drought_yearly = pd.read_parquet(os.path.join(processed_data_folder_name, drought_yearly_filename))
drought_schema_yearly = pd.read_csv(os.path.join(processed_data_folder_name, drought_yearly_shema), index_col=0).squeeze("columns")
df_drought_yearly = df_drought_yearly.astype(drought_schema_yearly.to_dict())

In [ ]:
df_decrees.head()

,insee,nom_commune,debut_evenement,fin_evenement,date_arrete,date_parution_jo,nom_peril,code_peril,franchise,code_nor,decision
43661,5063,LA GRAVE,2001-03-19,2001-03-25,2001-11-15,2001-12-01,Mouvement de Terrain,MVT,Simple,INTE0100649A,Reconnue
43662,5128,SAINT ANDRE D EMBRUN,2001-03-29,2001-03-29,2001-11-15,2001-12-01,Mouvement de Terrain,MVT,Simple,INTE0100649A,Reconnue
43663,6031,CANTARON,2000-11-23,2000-11-24,2001-11-15,2001-12-01,Mouvement de Terrain,MVT,Simple,INTE0100649A,Reconnue
43664,6031,CANTARON,2000-11-23,2000-11-24,2001-11-15,2001-12-01,Inondations et/ou Coulées de Boue,ICB,Simple,INTE0100649A,Reconnue
43665,6057,L'ESCARENE,2000-10-30,2000-10-31,2001-11-15,2001-12-01,Mouvement de Terrain,MVT,Simple,INTE0100649A,Reconnue


| Column    | Description                                      |
|-----------|--------------------------------------------------|
| date      | The date of the observation.                     |
| latitude  | The latitude coordinate of the observation point.|
| longitude | The longitude coordinate of the observation point.|
| number    | A unique identifier for the observation.         |
| expver    | Experiment version number.                       |
| u10       | 10-meter U-component of wind (eastward wind).    |
| v10       | 10-meter V-component of wind (northward wind).   |
| t2m       | 2-meter temperature (air temperature at 2 meters above the surface).|
| sp        | Surface pressure.                                |
| tp        | Total precipitation.                             |
| e         | Evaporation.                                     |
| sro       | Surface runoff.                                  |
| tcrw      | Total column water vapor.                        |
| stl1      | Soil temperature level 1.                        |
| stl2      | Soil temperature level 2.                        |
| slt       | Soil type.                                       |
| swvl1     | Volumetric soil water layer 1.                   |
| swvl2     | Volumetric soil water layer 2.                   |
| cvh       | High vegetation cover.                           |
| cvl       | Low vegetation cover.                            |
| tvh       | High vegetation type.                            |
| tvl       | Low vegetation type.                             |
